<a href="https://colab.research.google.com/github/MateusLauer/imglab/blob/master/tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15368, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15368 (delta 1), reused 4 (delta 0), pack-reused 15355
Receiving objects: 100% (15368/15368), 14.36 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (10504/10504), done.


In [3]:
!git clone https://github.com/onnx/onnx-tensorrt

Cloning into 'onnx-tensorrt'...
remote: Enumerating objects: 1648, done.
remote: Counting objects: 100% (721/721), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 1648 (delta 636), reused 519 (delta 519), pack-reused 927
Receiving objects: 100% (1648/1648), 2.19 MiB | 8.40 MiB/s, done.
Resolving deltas: 100% (1147/1147), done.


In [ ]:
!python yolov5/export.py --weights sku110k.pt --include engine --device 0

In [16]:
!pip install pycuda

NotImplementedError: ignored

In [9]:
import backend
import tensorrt as trt
import time
import numpy as np
import onnx
import torch

In [10]:
model_onnx = onnx.load('sku110k.onnx')
engine = backend.prepare(model_onnx, device='CUDA:0')


In [13]:
input_shapes = []
for input in model_onnx.graph.input:
    input_shapes.append(tuple(dim.dim_value for dim in input.type.tensor_type.shape.dim))

input_shape = input_shapes[0]

In [14]:

num_iterations = 1000
total_time = 0.0
with torch.no_grad():
    for i in range(num_iterations):
        input_data = torch.randn(input_shape).cuda()
        start_time = time.time()
        output_data = engine.run(input_data.cpu().numpy())[0]
        end_time = time.time()
        total_time += end_time - start_time
tensorrt_fps = num_iterations /total_time
tensorrt_fps = num_iterations / total_time
print(f"TensorRT FPS: {tensorrt_fps:.2f}")

TensorRT FPS: 126.97


In [15]:
!nvidia-smi

NotImplementedError: ignored